crm.deal.list для работы со сделками
crm.dealcategory.list категории сделок
crm.status.list Метод для получения справочника

In [ ]:
https://sputniksystems.bitrix24.ru/rest/2889/htzgpx8yzbqesi54/crm.dealcategory.list.json

In [ ]:
import requests

# Вебхук (замени на свой при необходимости)
WEBHOOK_URL = "https://sputniksystems.bitrix24.ru/rest/2889/7ydr3z1mpxni3pup/crm.dealcategory.list.json"

# Метод для получения справочника
method = "crm.status.list"

# Фильтр для выбора только типов сделок
params = {
    "filter": { "ENTITY_ID": "DEAL_TYPE" }
}

# Запрос
response = requests.post(WEBHOOK_URL + method, json=params)

if response.status_code == 200:
    data = response.json()
    print("Список всех TYPE_ID (типов сделок):")
    for item in data.get("result", []):
        print(f"{item['STATUS_ID']} → {item['NAME']}")
else:
    print("Ошибка:", response.status_code, response.text)


In [ ]:
import requests
import csv

WEBHOOK_URL = "https://sputniksystems.bitrix24.ru/rest/2889/htzgpx8yzbqesi54/"
CATEGORY_NAME = "Продажи оборудования"   # название направления, как в интерфейсе
OUTPUT_CSV = "deals_garantijnyj_remont.csv"

def call(method: str, payload: dict | None = None):
    resp = requests.post(WEBHOOK_URL + method, json=(payload or {}), timeout=60)
    resp.raise_for_status()
    data = resp.json()
    if "error" in data:
        raise RuntimeError(f"{data['error']}: {data.get('error_description')}")
    return data

# 1) Находим ID категории по названию
cats = call("crm.deal.category.list")["result"]
cat = next((c for c in cats if c.get("NAME") == CATEGORY_NAME), None)
if not cat:
    raise SystemExit(f"Категория '{CATEGORY_NAME}' не найдена. Доступные: {[c['NAME'] for c in cats]}")

category_id = int(cat["ID"])
print(f"Найдена категория '{CATEGORY_NAME}' с ID={category_id}")

# 2) Вытягиваем сделки этой категории (с пагинацией 'start')
fields = [
    "ID","TITLE","CATEGORY_ID","STAGE_ID","OPPORTUNITY","CURRENCY_ID",
    "ASSIGNED_BY_ID","CONTACT_ID","COMPANY_ID","DATE_CREATE","DATE_MODIFY"
]
all_deals = []
start = 0

while True:
    payload = {
        "filter": {"CATEGORY_ID": category_id},
        "select": fields,
        "order": {"DATE_CREATE": "DESC"},
        "start": start
    }
    data = call("crm.deal.list", payload)
    result = data.get("result", [])
    all_deals.extend(result)
    if "next" in data:
        start = data["next"]
    else:
        break

print(f"Всего сделок: {len(all_deals)}")

# 3) Сохраняем в CSV
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for d in all_deals:
        row = {k: d.get(k, "") for k in fields}
        writer.writerow(row)

print(f"Сохранено в файл: {OUTPUT_CSV}")


In [2]:
import requests
import csv

WEBHOOK_URL = "https://sputniksystems.bitrix24.ru/rest/2889/3869etbr219zfgb6/"
CATEGORY_NAME = "Гарантийный ремонт"
OUTPUT_CSV = "deals_garantijnyj_remont_all_fields.csv"
TIMEOUT = 60

def call(method: str, payload: dict | None = None):
    resp = requests.post(WEBHOOK_URL + method, json=(payload or {}), timeout=TIMEOUT)
    resp.raise_for_status()
    data = resp.json()
    if "error" in data:
        raise RuntimeError(f"{data['error']}: {data.get('error_description')}")
    return data

# 1) Найти CATEGORY_ID по имени
cats = call("crm.dealcategory.list")["result"]
cat = next((c for c in cats if c.get("NAME") == CATEGORY_NAME), None)
if not cat:
    raise SystemExit(f"Категория '{CATEGORY_NAME}' не найдена. Доступные: {[c['NAME'] for c in cats]}")
category_id = int(cat["ID"])
print(f"Категория '{CATEGORY_NAME}' найдена, ID={category_id}")

# 2) Получить полный список полей сделки (включая пользовательские)
fields_meta = call("crm.deal.fields")["result"]
# Поля для select: все системные + все UF_*
system_fields = [k for k in fields_meta.keys() if not k.startswith("UF_")]
user_fields = [k for k in fields_meta.keys() if k.startswith("UF_")]
# Гарантируем присутствие ключевых полей в начале (для удобства)
key_order = ["ID", "TITLE", "CATEGORY_ID", "STAGE_ID", "OPPORTUNITY", "CURRENCY_ID",
             "ASSIGNED_BY_ID", "CONTACT_ID", "COMPANY_ID", "DATE_CREATE", "DATE_MODIFY"]
ordered_system = key_order + [f for f in system_fields if f not in key_order]
select_fields = ordered_system + user_fields

print(f"Всего полей к выгрузке: {len(select_fields)}")

# 3) Выгрузка всех сделок категории с пагинацией
all_deals = []
start = 0
while True:
    payload = {
        "filter": {"CATEGORY_ID": category_id},
        "select": select_fields,  # полный список полей
        "order": {"DATE_CREATE": "DESC"},
        "start": start
    }
    data = call("crm.deal.list", payload)
    deals = data.get("result", [])
    all_deals.extend(deals)
    if "next" in data:
        start = data["next"]
    else:
        break

print(f"Выгружено сделок: {len(all_deals)}")

# 4) Сохранение в CSV (UTF-8)
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=select_fields)
    writer.writeheader()
    for d in all_deals:
        row = {k: d.get(k, "") for k in select_fields}
        writer.writerow(row)

print(f"Сохранено: {OUTPUT_CSV}")


Категория 'Гарантийный ремонт' найдена, ID=25
Всего полей к выгрузке: 158
Выгружено сделок: 14021
Сохранено: deals_garantijnyj_remont_all_fields.csv
